In [7]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=e76c12cc857b795e5ad3504bfa7ce771a399e8f28507525494cccf6375fb4d0c
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [8]:
# Import findspark and initialize.
import findspark
findspark.init()

In [9]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [13]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [14]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

In [15]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
SELECT YEAR(date) as year_sold,  ROUND(AVG(price),2) as Average_Price
FROM home_sales
WHERE bedrooms = 4
GROUP BY year_sold
ORDER BY year_sold """
         ).show()


+---------+-------------+
|year_sold|Average_Price|
+---------+-------------+
|     2019|     300263.7|
|     2020|    298353.78|
|     2021|    301819.44|
|     2022|    296363.88|
+---------+-------------+



In [16]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
SELECT date_built, ROUND(AVG(price),2) as  Average_Price
FROM home_sales
WHERE bedrooms = 4 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built"""
         ).show()


+----------+-------------+
|date_built|Average_Price|
+----------+-------------+
|      2010|    287529.97|
|      2011|    294656.14|
|      2012|    289533.13|
|      2013|    291870.89|
|      2014|    289935.92|
|      2015|     299517.5|
|      2016|    289090.11|
|      2017|    289100.17|
+----------+-------------+



In [17]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
SELECT date_built, bedrooms, bathrooms, floors, sqft_living, ROUND(avg(price), 2) as avg_price
FROM home_sales
WHERE bedrooms =3
AND bathrooms =3
AND floors=2
AND sqft_living >= 2000
GROUP BY date_built, bedrooms, bathrooms, floors, sqft_living """
         ).show()


+----------+--------+---------+------+-----------+---------+
|date_built|bedrooms|bathrooms|floors|sqft_living|avg_price|
+----------+--------+---------+------+-----------+---------+
|      2012|       3|        3|     2|       2165| 217432.5|
|      2010|       3|        3|     2|       2215| 145507.0|
|      2016|       3|        3|     2|       2111| 273989.0|
|      2010|       3|        3|     2|       2449| 339996.0|
|      2016|       3|        3|     2|       2326| 190670.0|
|      2012|       3|        3|     2|       2377| 370767.0|
|      2014|       3|        3|     2|       2389| 296233.0|
|      2013|       3|        3|     2|       2327| 219136.0|
|      2013|       3|        3|     2|       2377| 324162.0|
|      2013|       3|        3|     2|       2462| 208224.0|
|      2010|       3|        3|     2|       2313| 130941.0|
|      2012|       3|        3|     2|       2496| 145568.0|
|      2011|       3|        3|     2|       2025| 125151.0|
|      2013|       3|   

In [18]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
SELECT view , ROUND(AVG(price), 2) as  Average_Price
FROM home_sales
WHERE price >= 350000
GROUP BY view
ORDER BY view DESC""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|Average_Price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 1.441267490386963 seconds ---


In [19]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [20]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [21]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""
SELECT view, ROUND(AVG(price), 2) as  Average_Price
FROM home_sales
WHERE price >= 350000
GROUP BY view
ORDER BY view DESC""").show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|Average_Price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.5763235092163086 seconds ---


In [22]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("delayed_partitioned")

In [23]:
# 11. Read the formatted parquet data.
parquet_df = spark.read.parquet('delayed_partitioned')

In [24]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("home_sales_parquet")

In [25]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""
SELECT view, ROUND(avg(price), 2) as average_price
FROM home_sales_parquet
GROUP BY view
HAVING avg(price) >= 350000
SORT BY view DESC
""").show()



print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 1.3722293376922607 seconds ---


In [26]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [27]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False